In [57]:
import numpy as np

In [58]:
def index(i):
    """Calculate the step-value of the i-th Haar wavelet."""
    j = int(np.ceil(np.log2(i))) - 1
    k = int(i - 2**j) - 1
    return j, k

def collocation(N):
    return np.linspace(0, 1, N, endpoint=False) + 0.5 / N

def haar_int_2(x, i):
    """
    x: input vector
    i: the index of the Haar wavelet function

    return: the second-order integration of the Haar wavelet function
    """
    if i == 1:
        return 0.5 * x ** 2
    if i >= 2:
        j, k = index(i) # j is the scale, k is the translation
        m = 2 ** j
        alpha = k / m
        beta = (k + 0.5) / m
        gamma = (k + 1) / m
        a = 1. * (x>=alpha) * (x<beta) * (x - alpha) ** 2 / 2
        b = -1. * (x>=beta) * (x<=gamma) * ((x - gamma) ** 2 / 2 - (beta - alpha) ** 2)
        c = 1. * (x>= gamma) * (x <= 1) * (beta - alpha) ** 2
        if i != 0 and (i & (i - 1)) == 0: # if i is power of 2
            c = 0
        int_2 = a + b + c
        return int_2

def haar_int_2_mat(x, N):
    mat = np.zeros((N, len(x)))
    for j in range(1, N + 1):
        mat[:, j - 1] = haar_int_2(x, j)
    return mat


In [59]:
x = collocation(2)
t = collocation(2)
print(x)

[0.25 0.75]


In [60]:
f = lambda x: np.exp(x) + np.exp(-x)
K = lambda x, t: -np.exp(-(x + t))
u_true = lambda x: np.exp(x)

In [61]:
f = lambda x: 5/6 * x - 1/9
K = lambda x, t: 1/3 * (x + t)
u_true = lambda x: x

In [62]:
S2 = 1/2 * (K(0, t[0]) + K(0, t[1]))

S3 = 1/2 * (K(1, t[0]) + K(1, t[1]))

S4 = 1/2 * (K(0, t[0]) * t[0] 
            + K(0, t[1]) * t[1])

S5 = 1/2 * (K(1, t[0]) * t[0]
            + K(1, t[1]) * t[1])

S8 = 1 - S2 + S4 * (1 - S3) - S5 * (1 - S2)

S6 = np.zeros(2)
S6[0] = 1/2 * (K(0, t[0]) * haar_int_2(t[0], 1)
            + K(0, t[1]) * haar_int_2(t[1], 1))
S6[1] = 1/2 * (K(0, t[0]) * haar_int_2(t[0], 2)
            + K(0, t[1]) * haar_int_2(t[1], 2))

S7 = np.zeros(2)
S7[0] = 1/2 * (K(1, t[0]) * haar_int_2(t[0], 1)
            + K(1, t[1]) * haar_int_2(t[1], 1))
S7[1] = 1/2 * (K(1, t[0]) * haar_int_2(t[0], 2)
            + K(1, t[1]) * haar_int_2(t[1], 2))

A = f(0) * (1 - S5) + f(1) * S4

D = -f(0) * (1 - S3) + f(1) * (1 - S2)

In [63]:
VB = np.zeros(2)
VB[0] = haar_int_2(1, 1) - S7[0]
VB[1] = haar_int_2(1, 2) - S7[1]

VE = (1 - S5) * S6 - S4 * VB
VF = (1 - S3) * S6 + (1 - S2) * VB

VP = np.zeros(2)
VP[0] = 1 - 1/2 * (K(x[0], t[0]) + K(x[0], t[1]))
VP[1] = 1 - 1/2 * (K(x[1], t[0]) + K(x[1], t[1]))

VQ = np.zeros(2)
VQ[0] = x[0] - 1/2 * (K(x[0], t[0]) * t[0] + K(x[0], t[1]) * t[1])
VQ[1] = x[1] - 1/2 * (K(x[1], t[0]) * t[0] + K(x[1], t[1]) * t[1])

In [64]:
MA = np.zeros((2, 2))
MA[0, 0] = (
    K(x[0], t[0]) * haar_int_2(t[0], 1) + K(x[0], t[1]) * haar_int_2(t[1], 1)
) # i = 1
MA[0, 1] =  (
    K(x[0], t[0]) * haar_int_2(t[0], 2) + K(x[0], t[1]) * haar_int_2(t[1], 2)
) # i = 2
MA[1, 0] = (
    K(x[1], t[0]) * haar_int_2(t[0], 1) +  K(x[1], t[1]) * haar_int_2(t[1], 1)
)
MA[1, 1] = (
    K(x[1], t[0]) * haar_int_2(t[0], 2) + K(x[1], t[1]) * haar_int_2(t[1], 2)
)

haar_mat = np.zeros((2, 2))
haar_mat[0, 0] = haar_int_2(x[0], 1)
haar_mat[0, 1] = haar_int_2(x[0], 2)
haar_mat[1, 0] = haar_int_2(x[1], 1)
haar_mat[1, 1] = haar_int_2(x[1], 2)

MA = haar_mat - 1/2 * MA

print(haar_mat - haar_int_2_mat(x, 2))

[[0. 0.]
 [0. 0.]]


In [65]:
# print all the values
print("S2 = ", S2)
print("S3 = ", S3)
print("S4 = ", S4)
print("S5 = ", S5)
print("S6 = ", S6)
print("S7 = ", S7)
print("S8 = ", S8)
print("A = ", A)
print("D = ", D)
print("VB = ", VB)
print("VE = ", VE)
print("VF = ", VF)
print("VP = ", VP)
print("VQ = ", VQ)
print("MA = ", MA)

S2 =  0.16666666666666666
S3 =  0.49999999999999994
S4 =  0.10416666666666667
S5 =  0.2708333333333333
S6 =  [0.03645833 0.02864583]
S7 =  [0.08854167 0.0703125 ]
S8 =  0.6597222222222223
A =  -0.005787037037037021
D =  0.6574074074074076
VB =  [0.41145833 0.1796875 ]
VE =  [-0.01627604  0.00217014]
VF =  [0.36111111 0.1640625 ]
VP =  [0.75       0.58333333]
VQ =  [0.10416667 0.52083333]
MA =  [[-0.01822917 -0.0078125 ]
 [ 0.20572917  0.15885417]]


In [66]:
LHS = MA + (1 / S8) * np.outer(VP, VE) - (1 / S8) * np.outer(VQ, VF)
RHS = f(x) - 1 / S8 * (A * VP + D * VQ)

In [67]:
print("LHS = ", LHS)
print("RHS = ", RHS)

LHS =  [[-0.09375 -0.03125]
 [-0.09375  0.03125]]
RHS =  [-1.38777878e-17 -2.22044605e-16]


In [68]:
coefs = np.linalg.solve(LHS, RHS)
C1 = (1/S8) * (A + np.dot(VE, coefs))
C2 = (1/S8) * (D - np.dot(VF, coefs))

In [69]:
print("C1 = ", C1)
print("C2 = ", C2)
print("coefs = ", coefs)

C1 =  -0.00877192982456142
C2 =  0.9964912280701758
coefs =  [ 1.25825276e-15 -3.33066907e-15]
